**this is a template notebook for Assignment 4 on Recommendation. To get a 60 you will need to complete chapter 1.
    The template is also just an indication. You can add more cells if needed, and can of course delete this line**

# <span style ='background:yellow'>Recommendation system </span>


Author: <span style='background:yellow'>Stephen pangga</span><br>
Student number: <span style='background:yellow'>629860</span><br>
Date: <span style='background:yellow'>10/06/2022</span>

For this notebook to work you must have installed the following packages (usually via pip install *packageName*:
* numpy
* pandas
* **\<add other packages\>**

From these we will need the following libraries:

In [2]:
# enter here all those 'from .... import ....'
import pandas as pd


# 1. The Item-Based Recommender

We are going to build an item-based recommender for **<span style='background:yellow'>\<YOUR INPUT></span>**. In this recommender we could for example **<span style='background:yellow'>\<give a brief description of an example how your recommender would work></span>**.

To begin we'll need to load some data.

In [8]:
# enter code to load your example data file(s)userId,movieId,rating,timestamp
rating_df = pd.read_csv('ratings_small.csv', sep=',', names=['userId', 'movieId', 'rating', 'timestamp'])
rating_df.head()

,userId,movieId,rating,timestamp
0,userId,movieId,rating,timestamp
1,1,31,2.5,1260759144
2,1,1029,3.0,1260759179
3,1,1061,3.0,1260759182
4,1,1129,2.0,1260759185


**<span style='background:yellow'>\<explain a bit about the data file(s), if you have multiple and need to merge them, use the code cell below></span>**

In [11]:
# Optional depending on your dataset, enter code to select appropriate columns, and merge the files.
movie_dataset = pd.read_csv('movies_metadata.csv', sep=',')
movie_dataset.head()

#create a new dataframe focusing on movieId and movie name


#merge table


C:\Users\steph\AppData\Local\Temp\ipykernel_5160\1433907481.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_dataset = pd.read_csv('movies_metadata.csv', sep=',')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


We need one row per user, and his/her scores for each **<span style='background:yellow'>\<YOUR OBJECT></span>**. We can do this by creating a pivot table.

In [4]:
# code to create pivot table


We also need to take the counts into consideration so lets keep those into their own dataframe, together with the average score:

In [5]:
# code to make a dataframe with the unique objects, their average score and the number of scores


To decide on a cut-off point as minimum number of scores to be eligable we can plot the number of scores in a histogram:

In [6]:
# code to generate the histogram


From the histogram we can see that **<span style='background:yellow'>\<Explain in your own words what you notice and what you decide on as the cut-off point, explain why></span>**

Finally we can create the code to perform a recommendation. First we can select one particular **<span style='background:yellow'>\<YOUR OBJECT></span>**

In [7]:
# code to select one field


And then to find the recommendation based on the correlations.

In [8]:
# code to create the recommendation


**<span style='background:yellow'>\<interpret the result></span>**

It might be nice to have one simple function that could do all the work for us. It should have as input parameter a field and then returns the recommendation:

In [9]:
# code for a python function



Lets test the function:

In [10]:
# code to test the function


<font size="5"><center>-----Chapters 1 is required to be fully completed to get a 60, the next few chapters will give a +10 for each chapter.<br> 
    However no template is available for these chapters. You will have to create it yourself.
    ----</center></font>

# 2. Negative correlations

The recommenders so far used, are looking at what if I give a high rating for this, what do you then recommend me. But how about if I give a low rating for something, what would you then recommend me?

# 3. Content Based

Add a chapter showing how a basic content based recommender might work. A starting point for this might be https://www.datacamp.com/community/tutorials/recommender-systems-python (at about 1/3 of the page it starts explaining this in the section called ‘Content based’.

# 4. User Based

Add a chapter showing how a basic user based recommender might work. A starting point for this might be https://realpython.com/build-recommendation-engine-collaborative-filtering/

# 5. Combination

Combine two or more recommendation techniques to get an even better recommendation